In [ ]:
from pydrake.all import (
    RgbdSensor,
    DiagramBuilder,
    AddMultibodyPlantSceneGraph,
    Parser,
    ConnectDrakeVisualizer,
    FindResourceOrThrow,
    Simulator,
    RigidTransform,
    RollPitchYaw,
    DepthCameraProperties,
    MakeRenderEngineVtk, RenderEngineVtkParams,
)

In [ ]:
import os
import numpy as np

In [ ]:
def xyz_rpy_deg(xyz, rpy_deg):
    rpy_deg = np.asarray(rpy_deg)
    return RigidTransform(RollPitchYaw(rpy_deg * np.pi / 180), xyz)

In [ ]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder)

sdf_file = FindResourceOrThrow("drake/manipulation/models/iiwa_description/sdf/iiwa14_no_collision.sdf")
iiwa_1 = Parser(plant).AddModelFromFile(sdf_file, model_name="iiwa_1")
iiwa_2 = Parser(plant).AddModelFromFile(sdf_file, model_name="iiwa_2")

ConnectDrakeVisualizer(builder, scene_graph)

scene_graph.AddRenderer("renderer", MakeRenderEngineVtk(RenderEngineVtkParams()))
depth_prop = DepthCameraProperties(
    width=640, height=480, fov_y=np.pi/4, renderer_name="renderer",
    z_near=0.01, z_far=10.)

world_id = plant.GetBodyFrameIdOrThrow(plant.world_body().index())
X_WB = xyz_rpy_deg([4, 0, 0], [0, -90, 0])
sensor = RgbdSensor(world_id, X_PB=X_WB, properties=depth_prop)
builder.AddSystem(sensor)
builder.Connect(scene_graph.get_query_output_port(), sensor.query_object_input_port())

plant.Finalize()
diagram = builder.Build()
context = diagram.CreateDefaultContext()

In [ ]:
Simulator(diagram).Initialize()

In [ ]:
image = sensor.color_image_output_port().Eval(context)